In [1]:
from pystac import Catalog
from pathlib import Path
from shapely import Point, Polygon
import xarray as xr
from pyproj import Transformer

In [2]:
catalog_path = Path('./data/catalog/catalog.json')
catalog = Catalog.from_file(catalog_path)

In [3]:
example_stm = xr.open_zarr('./download/stm_example.zarr')
example_stm

<xarray.Dataset>
Dimensions:    (space: 78582, time: 10)
Coordinates:
    azimuth    (space) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
    lat        (space) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    lon        (space) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    range      (space) int64 dask.array<chunksize=(1000,), meta=np.ndarray>
  * time       (time) int64 0 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: space
Data variables:
    amplitude  (space, time) float32 dask.array<chunksize=(1000, 10), meta=np.ndarray>
    complex    (space, time) complex64 dask.array<chunksize=(1000, 10), meta=np.ndarray>
    phase      (space, time) float32 dask.array<chunksize=(1000, 10), meta=np.ndarray>
Attributes:
    multi-look:  coarsen-mean

In [4]:
bbox_stm_wgs84 = [
    example_stm["lon"].values.min(),
    example_stm["lat"].values.min(),
    example_stm["lon"].values.max(),
    example_stm["lat"].values.max(),
]
bbox_stm_wgs84

[4.7981586, 52.250504, 4.976751, 52.470882]

In [5]:
# Transform to RD
transformer = Transformer.from_crs("EPSG:4326", "EPSG:28992")
bbox_stm_rd = [
    *transformer.transform(bbox_stm_wgs84[1], bbox_stm_wgs84[0]),
    *transformer.transform(bbox_stm_wgs84[3], bbox_stm_wgs84[2])
]
bbox_stm_rd

[114775.61921292896, 473769.9741285362, 127109.96237056024, 498206.0458581245]

In [6]:
# Create search polygon from bounding box
polygon_search = Polygon(
    [
        (bbox_stm_rd[0], bbox_stm_rd[1]),
        (bbox_stm_rd[0], bbox_stm_rd[3]),
        (bbox_stm_rd[2], bbox_stm_rd[3]),
        (bbox_stm_rd[2], bbox_stm_rd[1]),
    ]
)

In [7]:
# Iterate over the items in the catalog
items = []
for item in catalog.get_all_items():
    # Check if the item intersects with the point
    bbox = item.bbox
    polygon = Polygon([(bbox[0], bbox[1]), (bbox[0], bbox[3]), (bbox[2], bbox[3]), (bbox[2], bbox[1])])
    if polygon_search.intersects(polygon):
        items.append(item)
items

[<Item id=bag>]

In [8]:
# Get the data path
# This path can be used in stmtools.stm.enrich_from_polygon() function to perform enrichment
data_paths =  [Path(item.assets['data'].href) for item in items]
data_paths

[PosixPath('/home/oku/Developments/MobyLe/example_scripts/stac_catalog/data/bag.parquet')]